In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sms=pd.read_csv("/content/Spam_SMS.csv")

In [ ]:
sms.head(5)

In [ ]:
sms.tail()

In [ ]:
sms.shape

In [ ]:
sms.isnull().sum()

In [ ]:
sms.duplicated().sum()

In [ ]:
sms.drop_duplicates(inplace=True)

In [ ]:
sms.shape

In [ ]:
data=sms.copy()

In [ ]:
data.shape

In [ ]:
data['Class'] = data['Class'].replace({'ham': 0, 'spam': 1})


In [ ]:
data.head()

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report ,accuracy_score,confusion_matrix
import pickle
import warnings
warnings.filterwarnings("ignore")
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#printing the stopwords in English
print(stopwords.words('english'))

In [ ]:

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatizing(content):
    # Remove noise (URLs, mentions, etc.)
    content = re.sub(r'http\S+|www\S+|https\S+', '', content, flags=re.MULTILINE)
    content = re.sub(r'@\w+', '', content)
    content = re.sub('[^a-zA-Z]', ' ', content)

    content = content.lower()
    content = content.split()
    content = [lemmatizer.lemmatize(word) for word in content if word not in stopwords.words('english')]
    return ' '.join(content)

data["Message"] = data["Message"].apply(lemmatizing)


In [ ]:
#separating the data and label
X = data['Message'].values
y = data['Class'].values

In [ ]:
# convert the textual data to Feature vectors
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'solver': ['liblinear', 'saga'],  # Solver to use
}

grid = GridSearchCV(LogisticRegression(), param_grid, scoring='accuracy', cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
print(f"Best Parameters: {grid.best_params_}")

# Predict and evaluate on the validation set
y_pred = best_model.predict(X_val)
print(classification_report(y_val, y_pred))
accuracy_sklearn = accuracy_score(y_val, y_pred)
print(f"Improved Accuracy using GridSearchCV: {accuracy_sklearn:.2f}")


In [ ]:
data['Class'] = data['Class'].replace({'ham': 0, 'spam': 1})

# Function to predict label for new content
def predict_label(text):
    # Preprocess the input text
    processed_text = lemmatizing(text)
    # Transform the text using the fitted vectorizer
    text_vector = vectorizer.transform([processed_text])
    # Predict the label
    predicted_label = best_model.predict(text_vector)
    # Map the numerical label to the corresponding emotional label
    return label_mapping[predicted_label[0]]

# Example of predicting a new input
new_input_text = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
predicted_label = predict_label(new_input_text)
print(f"Predicted Label: {predicted_label}")


In [ ]:
# Save the model
with open('best_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

# Save the vectorizer
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
